<a href="https://colab.research.google.com/github/jingwen0/systemeAQ/blob/main/analyse_sentiments_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
from numpy import unicode
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from gensim.models import KeyedVectors, Word2Vec
from sklearn import model_selection,preprocessing
import re
import unicodedata
import csv
import pandas as pd
import numpy as np
import textblob
import string
import time
import json
import pprint
from itertools import chain
import nltk
from nltk import word_tokenize

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.unicode` is a deprecated alias for `np.compat.unicode`. To silence this warning, use `np.compat.unicode` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `str` for which `np.compat.unicode` is itself an alias. Doing this will not modify any behaviour and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [5]:
def seg_sentence(data):
    # seg_words = word_tokenize(data, language="french")
    seg_words = data.split(" ")# 空格分词

    return seg_words

In [7]:
def read_csv(filename):
    # 加载数据
    df = pd.read_csv(filename, sep=',',error_bad_lines=False)
    print(df.head())
    print(df.shape)
    print(list(df.columns))
    text = df['text']
    label = df['label']
    print("总条数：",len(label),'\t',len(text))
    return text, label

In [7]:
!mkdir data
!wget -P data https://raw.githubusercontent.com/jingwen0/systemeAQ/main/corpus_mai.csv

--2022-06-07 20:33:14--  https://raw.githubusercontent.com/jingwen0/systemeAQ/main/corpus_mai.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 746317 (729K) [text/plain]
Saving to: ‘data/corpus_mai.csv’

corpus_mai.csv      100%[===================>] 728.83K  --.-KB/s    in 0.03s   

2022-06-07 20:33:15 (23.1 MB/s) - ‘data/corpus_mai.csv’ saved [746317/746317]



In [94]:
def cleaning_text(text):
    #Input: list of sentences
    #Output: list of sentences
    output = [re.sub('  ', ' ', x.lower()) for x in text]
    return output

In [96]:
# 获取原始数据
filename = './data/corpus_mai.csv'
texts, labels = read_csv(filename)

# 数据预处理工作
clean_texts = cleaning_text(list(texts))#分词

     id                                                               text  \
0  7101     Mon trésor, mon trésor ! On va se faire bouffer les couilles !   
1  6114  Julian, ici toi père. Même si en ce moment, basé sur les faits...   
2   438          Depuis la prison, il est encore plus méfiant et farouche.   
3  1904                  J'ai fait ma part. Je veux l'argent. Pas ce soir.   
4  2299                  Non. Eh bien, croyez-moi, je sais ce que je fais.   

   label  
0      1  
1      0  
2     -1  
3     -1  
4      1  
(9988, 3)
['id', 'text', 'label']
总条数： 9988 	 9988


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
def data_process(old_texts,texts, labels):
    seg_sentences = []
    new_labels = []
    sentences = []
    for i in range(len(labels)):
        data = texts[i]
        label = int(labels[i])
        old_data = old_texts[i]
        if label == 1 or  label == -1 or label == 0:

            word_tokens = seg_sentence(data)# 分词
            
            sentences.append(data)
            seg_sentences.append(word_tokens)
            new_labels.append(label)
        else:
            continue
    return seg_sentences,sentences,new_labels

In [103]:
seg_sentences,sentences,new_labels = data_process(texts, clean_texts, labels)
print(new_labels[0],'\t',sentences[0],'\t',seg_sentences[0])

1 	 mon trésor, mon trésor ! on va se faire bouffer les couilles ! 	 ['mon', 'trésor,', 'mon', 'trésor', '!', 'on', 'va', 'se', 'faire', 'bouffer', 'les', 'couilles', '!']


In [108]:
# 切分数据
ids = [i for i in range(len(new_labels))]
train_x_id, valid_x_id, train_y, valid_y = model_selection.train_test_split(ids, new_labels, test_size=0.1, random_state=5) #random_state为空时说明每次随机切分都不一样。测试时要固定测试集，随便设置一个数值。最好的办法还是一开始就把训练集和测试集分别写不同文件，永久固定
sentences = np.array(sentences)
train_x = sentences[np.array(train_x_id)]
valid_x = sentences[np.array(valid_x_id)]

seg_sentences = np.array(seg_sentences)
train_seg_x = seg_sentences[np.array(train_x_id)]
valid_seg_x = seg_sentences[np.array(valid_x_id)]
print(len(train_x))
print(len(valid_x))

8989
999


In [109]:
def encode_samples(tokenized_samples, word_to_idx,unk_id):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(unk_id)
        features.append(feature)
    return features

In [110]:
def pad_samples(features, maxlen=500, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features

In [111]:
def update_vector(weight, wvmodel, word_to_idx):
    # wv_size = len(wvmodel)#wv词表:155562
    # print('wv_size:',wv_size)

    # wv_words = wvmodel.key_to_index
    wv_words = wvmodel.index2word
    
    unk_count = 0
    
    for word,index in word_to_idx.items():

        if word in wv_words:
        
            vector = np.array(wvmodel.word_vec(word))
            weight[index, :] = torch.from_numpy(vector)
            #print('++++++++',word, index)
        else:
            #print("~~~~UNK:", word, index) 
            vector = weight[1]#UNK
            weight[index, :] = vector
            unk_count+=1
    print('unk_count:',unk_count)# UNK1888

    return weight

In [16]:
class LSTMNet(nn.Module):
    # 初始化，准备工作
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, weight, labels, use_gpu, **kwargs):
        super(LSTMNet, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.use_gpu = use_gpu
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.encoder = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                               num_layers=num_layers, bidirectional=self.bidirectional,
                               dropout=0)
        if self.bidirectional:
            self.decoder = nn.Linear(num_hiddens * 4, labels)
        else:
            self.decoder = nn.Linear(num_hiddens * 2, labels)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([1, 0, 2]))
        encoding = torch.cat([states[0], states[-1]], dim=1)
        outputs = self.decoder(encoding)
        return outputs

In [17]:
import os
def train(vocab_size, wvmodel, word_to_idx, train_features, train_labels, test_features, test_labels):

    # 设置超参数
    num_epochs = 20 # 数据跑几次
    embed_size = wvmodel.vector_size # 300维
    print('embed_size:',embed_size)
    num_hiddens = embed_size
    num_layers = 2
    bidirectional = True
    batch_size = 2 # 一次两个句子
    labels = 3
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    #lr = 0.8
    lr = 1e-3 # 0.001学习率
    device = torch.device("cpu:0")
    gpu_id = -1
    device = torch.device('cuda:{}'.format(gpu_id) if torch.cuda.is_available() and gpu_id>=0 else torch.device('cpu'))

    use_gpu = False
    
    # 使用词向量来初始化词表的表征为，10687=全量词袋
    weight = torch.rand(vocab_size+2, embed_size)# [10687+2, 200]
    # print(weight[2])# 检查是否更新
    weight = update_vector(weight, wvmodel, word_to_idx)
    #print(weight[2])
    print('weight.shape:',weight.shape)
    
    # 初始化模型，可替换
    net = LSTMNet(vocab_size=(vocab_size+2), embed_size=embed_size,
                   num_hiddens=num_hiddens, num_layers=num_layers,
                   bidirectional=bidirectional, weight=weight,
                   labels=labels, use_gpu=use_gpu)
    device = net.to(device)
    print(device)
    exit()
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr)
    
    # 加载数据，并随机打乱训练集
    train_set = torch.utils.data.TensorDataset(train_features, train_labels)
    test_set = torch.utils.data.TensorDataset(test_features, test_labels)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False)
    
    # 开始训练
    for epoch in range(num_epochs):
        start = time.time()
        train_loss, test_losses = 0, 0
        train_acc, test_acc = 0, 0
        n, m = 0, 0
        for feature, label in train_iter:
            n += 1
            #net.train()
            net.zero_grad()
            feature = Variable(feature.cuda())
            label = Variable(label.cuda())
            score = net(feature)
            loss = loss_function(score, label)
            loss.backward()
            #scheduler.step()
            optimizer.step()
            train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                     dim=1), label.cpu())
            train_loss += loss
        with torch.no_grad():
            for test_feature, test_label in test_iter:
                m += 1
                #net.eval()
                test_feature = test_feature.cuda()
                test_label = test_label.cuda()
                test_score = net(test_feature)
                test_loss = loss_function(test_score, test_label)
                test_acc += accuracy_score(torch.argmax(test_score.cpu().data,
                                                        dim=1), test_label.cpu())
                test_losses += test_loss
        end = time.time()
        runtime = end - start
        print('epoch: %d, train loss: %.4f, train acc: %.2f, test loss: %.4f, test acc: %.2f, time: %.2f' %
              (epoch, train_loss.data / n, train_acc / n, test_losses.data / m, test_acc / m, runtime))


In [65]:
!wget -P data https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/model_26.txt

--2022-06-07 22:25:03--  https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/model_26.txt
Resolving git.unistra.fr (git.unistra.fr)... 130.79.254.48
Connecting to git.unistra.fr (git.unistra.fr)|130.79.254.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17389513 (17M) [text/plain]
Saving to: ‘data/model_26.txt’

model_26.txt        100%[===================>]  16.58M  8.87MB/s    in 1.9s    

2022-06-07 22:25:06 (8.87 MB/s) - ‘data/model_26.txt’ saved [17389513/17389513]



In [112]:
embeddings_index = {}
with open('data/model_26.txt', 'r', encoding='utf8') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, "f", sep=" ")
      embeddings_index[word] = coefs

print(f'{len(embeddings_index)} vecteurs de mots ont été lus')

6082 vecteurs de mots ont été lus


In [113]:
# 获取训练集的词表
vocab = set(chain(*train_seg_x))
vocab_size = len(vocab)
#print(vocab)
print('vocab_size:',vocab_size)#10581,训练集词表太小了,测试集UNK会很多


vocab_size: 17152


In [115]:
# 构建查表字典 id:word, word:id   pad = 0 unk = 1
pad_id = 0
unk_id = 1
word_to_idx = {word: i+2 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = unk_id
idx_to_word = {i+2: word for i, word in enumerate(vocab)}
idx_to_word[unk_id] = '<unk>'

In [116]:
num_tokens = len(vocab)
embedding_dim = 300
hits = 0
misses = 0

# Préparation de la matrice
# Les mots qui ne se trouvent pas dans les plongements pré-entraînés seront 
# représentés par des vecteurs dont toutes les composantes sont égales à 0,
# y compris la représentation utilisée pour compléter les documents courts et
# celle utilisée pour les mots inconnus [UNK]
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_to_idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print(f'{hits} mots ont été trouvés dans les plongements pré-entraînés')
print(f'{misses} sont absents')

547 mots ont été trouvés dans les plongements pré-entraînés
16606 sont absents


In [30]:
# 句子转换成id，并补齐长度
max_lenth = 20
train_ids = encode_samples(train_seg_x, word_to_idx, unk_id) # unk为1
train_features = torch.tensor(pad_samples(train_ids, max_lenth, pad_id))# pad为0
train_labels = torch.tensor(train_y)
    
valid_ids = encode_samples(valid_seg_x, word_to_idx, unk_id) # unk为1
valid_features = torch.tensor(pad_samples(valid_ids, max_lenth, pad_id))# pad为0
valid_labels = torch.tensor(valid_y)
print(train_ids[0])
print(train_features[0])
print(train_features.shape) #[9000, 50]
print(valid_features.shape) #[1000, 50]

[4201, 7324, 5819, 9529, 1687, 6413, 6845, 7313, 6617, 9047, 8525, 2, 0, 0, 0, 0, 0, 0, 0, 0]
tensor([4201, 7324, 5819, 9529, 1687, 6413, 6845, 7313, 6617, 9047, 8525,    2,
           0,    0,    0,    0,    0,    0,    0,    0])
torch.Size([8989, 20])
torch.Size([999, 20])


In [23]:
 # 训练
train(vocab_size, word2vec_model, word_to_idx, train_features, train_labels, valid_features, valid_labels)

embed_size: 300
unk_count: 3236
weight.shape: torch.Size([10583, 300])
LSTMNet(
  (embedding): Embedding(10583, 300)
  (encoder): LSTM(300, 300, num_layers=2, bidirectional=True)
  (decoder): Linear(in_features=1200, out_features=3, bias=True)
)


RuntimeError: ignored

In [119]:
num_tokens = len(vocab)
embedding_dim = 300
hits = 0
misses = 0
EMBEDDING_DIM = 32
MAX_LENGTH = 20
PADDING_TYPE = "post"
TRUNC_TYPE = "post"
OUTPUT_SHAPE = len(set(train_labels))

In [123]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
import seaborn as sns
sns.set_theme(style="darkgrid")

In [129]:
# Listes utilisées pour sauvegarder les résultats obtenus à chaque pli
acc_per_fold = []
loss_per_fold = []
histories = []
folds = 50
stratkfold = model_selection.StratifiedKFold(n_splits=folds, shuffle=True, 
                                             random_state=12)
fold_no = 1
for train, test in stratkfold.split(train_x, train_y):
  # Création du modèle
  int_sequences_input = keras.Input(shape=(None,), dtype="int64")
  embedding_layer = Embedding(num_tokens, embedding_dim, trainable=True,
      embeddings_initializer=keras.initializers.Constant(embedding_matrix),
  )
  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Conv1D(64, 5, activation="relu")(embedded_sequences)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(64, 5, activation="relu")(x)
  x = layers.GlobalMaxPooling1D()(x)
  x = layers.Dense(64, activation="relu")(x)
  x = layers.Dropout(0.5)(x)
  preds = layers.Dense(len(class_names), activation="softmax")(x)
  model = keras.Model(int_sequences_input, preds)
  model.summary()
  

  print('------------------------------------------------------------------------')
  print(f'Entraînement pour le pli {fold_no} ...')
  fold_x_train = vectorizer(X_train.iloc[train].to_numpy()).numpy()
  fold_x_val = vectorizer(X_train.iloc[test].to_numpy()).numpy()
  fold_y_train = y_train.iloc[train].to_numpy()
  fold_y_val = y_train.iloc[test].to_numpy()

  # Compilation du modèle : permet de préciser la fonction de perte et l'optimiseur
  # loss=sparse_categorical_crossentropy : entropie croisée, dans le cas où les 
  #  classes cibles sont indiquées sous forme d'entiers. Il s'agira de minimiser
  #  la perte pendant l'apprentissage
  # optimizer=rmsprop : l'optimiseur détermine la manière doit les poids seront
  #  mis à jour pendant l'apprentissage
  model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
  )
  # Entraînement
  history = model.fit(fold_x_train, fold_y_train, batch_size=128, epochs=10, 
            validation_data=(fold_x_val, fold_y_val))
  histories.append(history)
  # Evaluation sur les données de validation
  scores = model.evaluate(fold_x_val, fold_y_val, verbose=0)
  print(f'Scores pour le pli {fold_no}: {model.metrics_names[0]} = {scores[0]:.2f};',
        f'{model.metrics_names[1]} = {scores[1]*100:.2f}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  fold_no = fold_no + 1

# Affichage des scores moyens par pli
print('---------------------------------------------------------------------')
print('Scores par pli')
for i in range(0, len(acc_per_fold)):
  print('---------------------------------------------------------------------')
  print(f'> Pli {i+1} - Loss: {loss_per_fold[i]:.2f}',
        f'- Accuracy: {acc_per_fold[i]:.2f}%')
print('---------------------------------------------------------------------')
print('Scores moyens pour tous les plis :')
print(f'> Accuracy: {np.mean(acc_per_fold):.2f}',
      f'(+- {np.std(acc_per_fold):.2f})')
print(f'> Loss: {np.mean(loss_per_fold):.2f}')
print('---------------------------------------------------------------------')

NameError: ignored

In [127]:
model_rnn = Sequential([
        embedded_sequences,
        Dropout(0.2),
        Conv1D(filters = 256, kernel_size = 3, activation = "relu"),
        MaxPooling1D(pool_size = 3),
        Conv1D(filters = 128, kernel_size = 3, activation = "relu"),
        MaxPooling1D(pool_size = 3),
        LSTM(128),
        Dense(128, activation = "relu"),
        Dropout(0.2),
        Dense(64, activation = "relu"),
        Dense(300, activation = "softmax")
])


TypeError: ignored

In [33]:
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 32)            338592    
                                                                 
 dropout_2 (Dropout)         (None, 20, 32)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 18, 256)           24832     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 6, 256)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 4, 128)            98432     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 1, 128)           0         
 1D)                                                    

In [34]:
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model_rnn.compile(loss = "sparse_categorical_crossentropy",
              optimizer = OPTIMIZER,
              metrics = ["accuracy"])

In [43]:
train_features_tf = tf.constant(train_features,dtype = tf.float32)
train_labels_tf = tf .constant(train_labels,dtype = tf.float32)
valid_features_tf = tf.constant(valid_features,dtype = tf.float32)
valid_labels_tf = tf .constant(valid_labels,dtype = tf.float32)

In [44]:
history = model_rnn.fit(train_features_tf, train_labels_tf,
                    epochs = 150,
                    validation_data = (valid_features_tf, valid_labels_tf),
                    batch_size = 64)

Epoch 1/150
141/141 [==============================] - 17s 13ms/step - loss: nan - accuracy: 0.3024 - val_loss: nan - val_accuracy: 0.2863
Epoch 2/150
141/141 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.3046 - val_loss: nan - val_accuracy: 0.2863
Epoch 3/150
141/141 [==============================] - 1s 9ms/step - loss: nan - accuracy: 0.3046 - val_loss: nan - val_accuracy: 0.2863
Epoch 4/150
141/141 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.3046 - val_loss: nan - val_accuracy: 0.2863
Epoch 5/150
141/141 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.3046 - val_loss: nan - val_accuracy: 0.2863
Epoch 6/150
141/141 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.3046 - val_loss: nan - val_accuracy: 0.2863
Epoch 7/150
141/141 [==============================] - 1s 8ms/step - loss: nan - accuracy: 0.3046 - val_loss: nan - val_accuracy: 0.2863
Epoch 8/150
141/141 [==================

KeyboardInterrupt: ignored

In [ ]:
embedding_dim = 300

model_cnn = Sequential()
model_cnn.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights = [embedding_matrix], input_length=maxlen))
model_cnn.add(layers.Dropout(0.3)) 
model_cnn.add(layers.Conv1D(filters=32, kernel_size=3, activation=act)) 
model_cnn.add(layers.MaxPool1D(pool_size=3)) 
model_cnn.add(layers.Dropout(0.3))
model_cnn.add(layers.Conv1D(filters=32, kernel_size=3, activation=act)) 
model_cnn.add(layers.MaxPool1D(pool_size=3)) 
model_cnn.add(layers.Dropout(0.3))
model_cnn.add(layers.Bidirectional(layers.LSTM(256, recurrent_dropout=0.3)))
model_cnn.add(layers.Dropout(0.3))
model_cnn.add(layers.Dense(256,activation=act)) 
model_cnn.add(layers.Dropout(0.3)) 
model_cnn.add(layers.Dense(6, activation="softmax"))
model_cnn.compile(optimizer=opt, loss="categorical_crossentropy",metrics=["accuracy"])
model_cnn.summary()